# How to use K-Means clustering to help with some of the exploratory aspects of customer segmentation.

In [18]:
import pandas as pd
pd.set_option("display.max_columns", 80)

In [10]:
sales_data = pd.read_csv("pcsg_cust_orders_export.csv", low_memory=False, parse_dates=["Paid at", "Fulfilled at", "Created at", "Cancelled at"])
# sales_data.columns = ["offer_id", "campaign", "varietal", "min_qty", "discount", "origin", "past_peak"]
# df_offers.head()

In [45]:
web_pos_serial = {'web': 1, 'pos': 2}
sales_data.Source = sales_data.Source.map(web_pos_serial)
sales_data['Risk Level'] = sales_data['Risk Level'].map({'Low': 1, 'Medium': 2, 'High': 3})
sales_data.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone
0,191569910468,c_chien_may@hotmail.com,paid,2017-09-28 05:33:13,unfulfilled,NaT,no,SGD,45.0,5.5,0.0,50.5,NaN,0.0,2 Working-Day Delivery,2017-09-28 05:33:12,1,Resist Skin Revealing Body Lotion 10% AHA (Gly...,45.0,NaN,5900,True,False,pending,May Chih,"72 bayshore rd , 26-16",72 bayshore rd,26-16,NaN,Singapore,'469988,NaN,SG,8288 7480,May Chih,"72 bayshore rd , 26-16",72 bayshore rd,26-16,NaN,Singapore,'469988,NaN,SG,8288 7480,NaN,NaN,NaT,Stripe,c16895272083.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,6.164888e+09,NaN,Low,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,191569914-1221,trymeepoh@hotmail.com,paid,2017-09-28 05:32:17,fulfilled,2017-09-28 05:32:17,yes,SGD,128.0,0.0,0.0,128.0,NaN,0.0,NaN,2017-09-28 05:32:16,1,Skin Perfecting 2% BHA (Salicylic Acid) Gel - ...,43.0,NaN,2040,True,False,fulfilled,Kim Chin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Stripe,c16895278931.1,0.0,Paula's Choice,0.0,Eli Goh,Front Porch,14.0,6.164888e+09,NaN,Low,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,191569914-1221,trymeepoh@hotmail.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-28 05:32:16,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,43.0,NaN,2010,True,False,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,191569914-1221,trymeepoh@hotmail.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-28 05:32:16,1,Skin Balancing Ultra-Sheer Daily Defense SPF 3...,42.0,NaN,1560,True,False,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,191569914-1220,lxy_cheryl@hotmail.com,paid,2017-09-28 04:44:29,fulfilled,2017-09-28 04:44:29,yes,SGD,38.0,0.0,0.0,38.0,Discount,5.0,NaN,2017-09-28 04:44:29,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,43.0,NaN,2010,True,False,fulfilled,Cheryl Leung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Stripe,c16895139091.1,0.0,Paula's Choice,0.0,Eli Goh,Front Porch,14.0,6.164835e+09,NaN,Low,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
sales_data.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone
0,191569910468,c_chien_may@hotmail.com,paid,2017-09-28 05:33:13,unfulfilled,NaT,no,SGD,45.0,5.5,0.0,50.5,NaN,0.0,2 Working-Day Delivery,2017-09-28 05:33:12,1,Resist Skin Revealing Body Lotion 10% AHA (Gly...,45.0,NaN,5900,True,False,pending,May Chih,"72 bayshore rd , 26-16",72 bayshore rd,26-16,NaN,Singapore,'469988,NaN,SG,8288 7480,May Chih,"72 bayshore rd , 26-16",72 bayshore rd,26-16,NaN,Singapore,'469988,NaN,SG,8288 7480,NaN,NaN,NaT,Stripe,c16895272083.1,0.0,Paula's Choice,0.0,NaN,NaN,NaN,6.164888e+09,NaN,Low,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,191569914-1221,trymeepoh@hotmail.com,paid,2017-09-28 05:32:17,fulfilled,2017-09-28 05:32:17,yes,SGD,128.0,0.0,0.0,128.0,NaN,0.0,NaN,2017-09-28 05:32:16,1,Skin Perfecting 2% BHA (Salicylic Acid) Gel - ...,43.0,NaN,2040,True,False,fulfilled,Kim Chin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Stripe,c16895278931.1,0.0,Paula's Choice,0.0,Eli Goh,Front Porch,14.0,6.164888e+09,NaN,Low,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,191569914-1221,trymeepoh@hotmail.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-28 05:32:16,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,43.0,NaN,2010,True,False,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,191569914-1221,trymeepoh@hotmail.com,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-28 05:32:16,1,Skin Balancing Ultra-Sheer Daily Defense SPF 3...,42.0,NaN,1560,True,False,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Paula's Choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,191569914-1220,lxy_cheryl@hotmail.com,paid,2017-09-28 04:44:29,fulfilled,2017-09-28 04:44:29,yes,SGD,38.0,0.0,0.0,38.0,Discount,5.0,NaN,2017-09-28 04:44:29,1,Skin Perfecting 2% BHA (Salicylic Acid) Liquid...,43.0,NaN,2010,True,False,fulfilled,Cheryl Leung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,Stripe,c16895139091.1,0.0,Paula's Choice,0.0,Eli Goh,Front Porch,14.0,6.164835e+09,NaN,Low,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
sales_cols = sales_data.columns[:]
sales_cols

Index(['Name', 'Email', 'Financial Status', 'Paid at', 'Fulfillment Status',
       'Fulfilled at', 'Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
       'Taxes', 'Total', 'Discount Code', 'Discount Amount', 'Shipping Method',
       'Created at', 'Lineitem quantity', 'Lineitem name', 'Lineitem price',
       'Lineitem compare at price', 'Lineitem sku',
       'Lineitem requires shipping', 'Lineitem taxable',
       'Lineitem fulfillment status', 'Billing Name', 'Billing Street',
       'Billing Address1', 'Billing Address2', 'Billing Company',
       'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country',
       'Billing Phone', 'Shipping Name', 'Shipping Street',
       'Shipping Address1', 'Shipping Address2', 'Shipping Company',
       'Shipping City', 'Shipping Zip', 'Shipping Province',
       'Shipping Country', 'Shipping Phone', 'Notes', 'Note Attributes',
       'Cancelled at', 'Payment Method', 'Payment Reference',
       'Refunded Amount', 'Vendor', '

In [54]:
sales_data_simple = sales_data[['Name', 'Email', 'Financial Status', 'Paid at', 'Fulfillment Status', 'Fulfilled at','Total']]
sales_data_simple.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Total
0,191569910468,c_chien_may@hotmail.com,paid,2017-09-28 05:33:13,unfulfilled,NaT,50.5
1,191569914-1221,trymeepoh@hotmail.com,paid,2017-09-28 05:32:17,fulfilled,2017-09-28 05:32:17,128.0
2,191569914-1221,trymeepoh@hotmail.com,NaN,NaT,NaN,NaT,NaN
3,191569914-1221,trymeepoh@hotmail.com,NaN,NaT,NaN,NaT,NaN
4,191569914-1220,lxy_cheryl@hotmail.com,paid,2017-09-28 04:44:29,fulfilled,2017-09-28 04:44:29,38.0


In [75]:
sales_data['NewColumn'] = sales_data['Subtotal'].apply(lambda x: x/2 if not pd.isnull(x) else 0)#  sales_data['Subtotal'] /2  # && sales_data['Paid at'] == np.NaT] 
sales_data[['Subtotal', 'NewColumn']]

,Subtotal,NewColumn
0,45.0,22.50
1,128.0,64.00
2,NaN,0.00
3,NaN,0.00
4,38.0,19.00
5,127.0,63.50
6,NaN,0.00
7,95.0,47.50
8,NaN,0.00
9,0.0,0.00


In [15]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20853 entries, 0 to 20852
Data columns (total 71 columns):
Name                           20853 non-null object
Email                          17134 non-null object
Financial Status               9464 non-null object
Paid at                        8967 non-null datetime64[ns]
Fulfillment Status             9464 non-null object
Fulfilled at                   9426 non-null datetime64[ns]
Accepts Marketing              9464 non-null object
Currency                       9464 non-null object
Subtotal                       9464 non-null float64
Shipping                       9464 non-null float64
Taxes                          9464 non-null float64
Total                          9464 non-null float64
Discount Code                  1891 non-null object
Discount Amount                9464 non-null float64
Shipping Method                4340 non-null object
Created at                     20853 non-null datetime64[ns]
Lineitem quantity          

## Clustering our customers
We can use their behavior (whether or not they purchased something based on an offer) as a way to group similar minded customers together. We can then study those groups to look for patterns and trends which can help us formulate future offers.

The first thing we need is a way to compare customers. To do this, we're going to create a matrix that contains each customer and a **0/1 indicator for whether or not they responded to a given offer.** This is easy enough to do in Python:

# join the offers and transactions table
df = pd.merge(df_offers, df_transactions)
# create a "pivot table" which will give us the number of times each customer responded to a given offer
matrix = df.pivot_table(index=['customer_name'], columns=['offer_id'], values='n')
# a little tidying up. fill NA values with 0 and make the index into a column
matrix = matrix.fillna(0).reset_index()
# save a list of the 0/1 columns. we'll use these a bit later
x_cols = matrix.columns[1:]

from sklearn.cluster import KMeans
cluster = KMeans(n_clusters = 5)
sales_data['cluster'] = cluster.fit_predict(sales_data[sales_data.columns[1:]])
sales_data.cluster.value_counts()

### Principal Component Analysis. 

To transform our multi-dimensional dataset into a 2 dimensional dataset. Once it is in 2 dimensions (or simply put, it has 2 columns), it becomes much easier to plot!

In [22]:
'''from sklearn.decomposition import PCA

pca = PCA(n_components=2)
matrix['x'] = pca.fit_transform(matrix[x_cols])[:,0]
matrix['y'] = pca.fit_transform(matrix[x_cols])[:,1]
matrix = matrix.reset_index()

customer_clusters = matrix[['customer_name', 'cluster', 'x', 'y']]
customer_clusters.head()'''

from sklearn.decomposition import PCA

In [36]:
pca= PCA(n_components=2)
sales_data['x'] = pca.fit_transform(sales_data[sales_cols])[:, 0]
sales_data['y'] = pca.fit_transform(sales_data[sales_cols])[:, 1]
sales_data_2 = sales_data.reset_index()

ValueError: could not convert string to float: 'pos'

In [60]:
sales_data_simple.shape

(20853, 7)